# Mathematically Model for Maximum Flow Problem Using Linear Programming

The maximum flow problem can be elegantly formulated as a linear
programming problem, providing both theoretical insights and practical
solution methods. This mathematical modeling approach transforms the
combinatorial flow optimization into a structured optimization
framework.

## Standard Linear Programming Formulation

The most direct formulation treats each edge flow as a decision
variable. For a flow network $G = (V, E)$ with source $s$, sink $t$, and
capacity function $c: E \rightarrow \mathbb{R}_0^+$, we define:

**Decision Variables:**

-   $f(u,v)$ = flow through edge $(u,v) \in E$

**Linear Program:** $\text{maximize} \sum_{v:(s,v) \in E} f(s,v)$

subject to:
$\sum_{u:(u,v) \in E} f(u,v) = \sum_{w:(v,w) \in E} f(v,w) \quad \forall v \in V - \{s,t\}$
$f(u,v) \leq c(u,v) \quad \forall (u,v) \in E$
$f(u,v) \geq 0 \quad \forall (u,v) \in E$

The objective function maximizes the total flow leaving the source
node[1][2]. The first constraint ensures **flow conservation** at all
intermediate nodes - the amount of flow entering each node equals the
amount leaving it[3]. The second constraint enforces **capacity limits**
on each edge, while the third ensures **non-negativity** of flows[4][5].

## Alternative Formulation with Artificial Arc

A mathematically elegant approach introduces an artificial arc from sink
back to source with infinite capacity. This allows uniform treatment of
all nodes under a single conservation constraint[6]:

**Modified Formulation:** $\text{maximize} \; x_{ts}$

subject to:
$\sum_{j:(i,j) \in E'} x_{ij} - \sum_{j:(j,i) \in E'} x_{ji} = 0 \quad \forall i \in V$
$0 \leq x_{ij} \leq c_{ij} \quad \forall (i,j) \in E$

where $E'$ includes the original edges plus the artificial arc
$(t,s)$[7]. This formulation treats the maximum flow as the flow through
the artificial return arc.

## Path-Based Formulation

An alternative perspective models flows in terms of paths rather than
individual edges[8]. Let $P$ denote the set of all simple paths from $s$
to $t$:

**Path Variables:**

-   $x_p$ = flow along path $p \in P$

**Path-Based Linear Program:** $\text{maximize} \sum_{p \in P} x_p$

subject to:
$\sum_{p \in P: (u,v) \in p} x_p \leq c(u,v) \quad \forall (u,v) \in E$
$x_p \geq 0 \quad \forall p \in P$

This formulation naturally eliminates flow conservation constraints
since each path variable represents a complete route from source to
sink[9]. However, it typically involves exponentially many variables
corresponding to all possible paths.

## Practical Implementation Example


In [1]:
import numpy as np
from scipy.optimize import linprog

def max_flow_lp(capacity_matrix, source, sink):
    """
    Solve maximum flow using linear programming
    """
    n = len(capacity_matrix)

    # Create decision variables for each edge (i,j)
    # Variables ordered as: x_01, x_02, ..., x_ij, ...
    edges = []
    edge_to_var = {}
    var_count = 0

    for i in range(n):
        for j in range(n):
            if capacity_matrix[i][j] > 0:
                edges.append((i, j))
                edge_to_var[(i, j)] = var_count
                var_count += 1

    # Objective: maximize sum of flows leaving source
    c = np.zeros(var_count)
    for i, (u, v) in enumerate(edges):
        if u == source:
            c[i] = -1  # Negative for maximization

    # Capacity constraints: f_ij <= c_ij
    A_ub = np.eye(var_count)
    b_ub = [capacity_matrix[u][v] for u, v in edges]

    # Flow conservation constraints
    A_eq = []
    b_eq = []

    for node in range(n):
        if node == source or node == sink:
            continue

        constraint = np.zeros(var_count)
        for i, (u, v) in enumerate(edges):
            if v == node:  # Flow into node
                constraint[i] = 1
            elif u == node:  # Flow out of node
                constraint[i] = -1

        A_eq.append(constraint)
        b_eq.append(0)

    A_eq = np.array(A_eq) if A_eq else None
    b_eq = np.array(b_eq) if b_eq else None

    # Solve LP
    bounds = [(0, None) for _ in range(var_count)]

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
                     bounds=bounds, method='highs')

    return -result.fun if result.success else None

# Example usage
capacity_matrix = [
    [0, 3, 2, 0],  # Source
    [0, 0, 1, 3],  # Node 1
    [0, 0, 0, 2],  # Node 2
    [0, 0, 0, 0]   # Sink
]

max_flow_value = max_flow_lp(capacity_matrix, 0, 3)
print(f"Maximum flow: {max_flow_value}")


Maximum flow: 5.0


## Theoretical Advantages

The linear programming formulation provides several theoretical
benefits[10]:

**Optimality Guarantees:** The LP solution yields the globally optimal
maximum flow, as the feasible region forms a convex polytope and the
objective function is linear[11].

**Duality Theory:** The dual of the maximum flow LP corresponds to the
minimum cut problem, providing the mathematical foundation for the
max-flow min-cut theorem[12]\[^5\].

**Sensitivity Analysis:** LP theory enables analysis of how changes in
capacities affect the optimal flow value through shadow prices and
reduced costs[13].

**Multiple Constraints:** The LP framework easily accommodates
additional constraints such as budget limitations, cost considerations,
or flow splitting requirements[14].

## Computational Considerations

While the LP formulation provides theoretical elegance, specialized
network flow algorithms like Ford-Fulkerson typically outperform
general-purpose LP solvers for pure maximum flow problems[15]. However,
the LP approach becomes advantageous when dealing with variants
involving additional constraints or when integrating flow optimization
into larger mathematical programming models.

The equivalence between the edge-based and path-based formulations
demonstrates the flexibility of linear programming in modeling the same
optimization problem from different perspectives, each offering unique
insights into the problem structure[16].

⁂

[1] http://www.cs.emory.edu/~cheung/Courses/253/Syllabus/NetFlow/max-flow-lp.html

[2] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[3] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[4] http://www.cs.emory.edu/~cheung/Courses/253/Syllabus/NetFlow/max-flow-lp.html

[5] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[6] https://www.youtube.com/watch?v=Pc9gE7psz6o

[7] https://www.youtube.com/watch?v=Pc9gE7psz6o

[8] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[9] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[10] https://www.pvamu.edu/mathematics/wp-content/uploads/sites/49/33_R1095_Ram_AAM_V13_1_pp_508_515_060118.pdf

[11] http://www.cs.emory.edu/~cheung/Courses/253/Syllabus/NetFlow/max-flow-lp.html

[12] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf

[13] https://www.pvamu.edu/mathematics/wp-content/uploads/sites/49/33_R1095_Ram_AAM_V13_1_pp_508_515_060118.pdf

[14] https://www.pvamu.edu/mathematics/wp-content/uploads/sites/49/33_R1095_Ram_AAM_V13_1_pp_508_515_060118.pdf

[15] https://www.pvamu.edu/mathematics/wp-content/uploads/sites/49/33_R1095_Ram_AAM_V13_1_pp_508_515_060118.pdf

[16] https://theory.stanford.edu/~trevisan/cs261/lecture15.pdf